In [ ]:
import sagemaker as sage
from time import gmtime, strftime
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

In [ ]:
sess = sage.Session()

In [ ]:
hyperparameters = {
    "task" : "tasd", 
    "dataset" : "dataset", 
    "model_name_or_path" : "t5-base", 
    "paradigm": "extraction",
    "eval_batch_size" :"16",
    "train_batch_size" :"2",
    "learning_rate" :"3e-4",
    "num_train_epochs":"1",
    "n_gpu": "1"
}

In [ ]:
entry_point = 'finetune.py'
source_dir = 's3://all-in-one-ai-demo-assets-ap-southeast-1/datalab/absa/sourcedir.tar.gz'
git_config = None
role = get_execution_role()
framework_version = '1.7.1'
py_version='py36'
instance_type='ml.p3.2xlarge'
instance_count=1

In [ ]:
estimator = PyTorch(
    entry_point = entry_point,
    source_dir = source_dir,
    git_config = git_config,
    role = role,
    debugger_hook_config=False,
    hyperparameters = hyperparameters,
    framework_version = framework_version, 
    py_version = py_version,
    instance_type = instance_type,
    instance_count = instance_count
)

In [ ]:
inputs = {'dataset': 's3://all-in-one-ai-demo-assets-ap-southeast-1/datalab/absa/data/'}

In [ ]:
response = estimator.fit(inputs)

In [ ]:
import sagemaker

instance_type = 'ml.m5.4xlarge'
role = sagemaker.get_execution_role()

In [ ]:
from sagemaker.pytorch.model import PyTorchModel

pytorch_model = PyTorchModel(model_data='s3://sagemaker-ap-southeast-1-116572824542/pytorch-training-2022-05-28-10-05-39-029/output/model.tar.gz', 
                             role=role,
                             entry_point='inference.py', 
                             source_dir='s3://all-in-one-ai-demo-assets-ap-southeast-1/datalab/absa/sourcedir.tar.gz', 
                             framework_version='1.7.1', 
                             py_version='py36'
                ) # TODO set model_server_workers=1 to avoid torchhub bug

predictor = pytorch_model.deploy(instance_type=instance_type, initial_instance_count=1)

In [ ]:
predictor.predict({"inputs": "Worth an hour of frustration to put together Although not the easiest product I’ve ever assembled, it was worth the few minutes of cursing. I have not decided where I will put the lamp but I’m glad I purchased it. Had to struggle with one that was lost but Amazon made that right by sending another. Others have complained about the shade but that was very simple to put together. It looks like a quality item especially when compared to floor lamps I’ve seen on the floor at stores like Home Goods or Lowes. I’m happy with it. "}, initial_args={'ContentType': 'application/json'})